# APD Demographics data pipeline

This story started with the receipt of a dozen Excel files, in which each file had eight worksheets. I needed to combine all the data from all the worksheets into a single file I could then analyze (using Tableau.) If I did this by hand, I would have to open each file and copy/paste each worksheet. That's almost 100 copy/pastes, and it was very likely I would mess something up along the way.

So I made a bash shell script that uses [csvkit](https://csvkit.readthedocs.org) to do the work for me. We'll replicate that here.

We will:
- download all the files using `curl` into our working directory
- use csvkit commands to extract each sheet into a csv file
- use csvkit to combine them into a single file

(So you know, after I combined the file, I did some cleaning in Open Refine before visualizting in Tableau.)

Of note: the speadsheet names that we need are:

- AsstChief
- Cmdr
- Lt
- Sgt
- Det
- Corp
- PO
- Cadets


## Create directory to download data

By this point, we should have already created our class directory inside of our home directory. Our next step is to create the folder where we will pull down our data.

In [2]:
# type these commands into your notebook
# include this note with # at beginning:

# set up folders if not there already
# go to home folder, then Documents
cd ~/Documents/rwd/

# create apd inside class directory
mkdir -p apd/

# creeate data inside that directory
mkdir -p apd/data/

# create data-done directory
mkdir -p apd/data-done/

# go inside the data directory
cd apd/data/

# print working directory to make sure you are in it
pwd

/Users/christian/Documents/rwd/apd/data


## Getting the data

We'll use a new command called `curl` to download our data. For more information on curl, you can read the [man page](https://curl.haxx.se/docs/manpage.html) or this [handy tip sheet](http://www.thegeekstuff.com/2012/04/curl-examples/), which is much more understandable.

Since we are in a Bash notebook, we can use our command-line tools. We are going to pull down 12 files of a similar name. Before we do, let's break down the `curl` statement below.

* `curl` is the command. I think of it as "capture URL". [man curl](http://man.cx/curl)
* `-O` (that's capital O, not zero). This outputs result to a file to your computer instead of to your screen, using the same file name as it was originally.
* `-L` stands for `--location`, and it will allow the request to follow a redirect link. It's good to use it.
* And then we have the url of the file.
  * We did something special in the URL to the file. There are 12 files, named 2005.xls, 2006.xls, etc., up to 2016.xls. We used brackets to give `curl` a series of filenames to process all at once. This works because the filenames are sequential.


In [3]:
# curl command to download files from my github repo
curl -O -L https://raw.githubusercontent.com/utdata/cli-tools/master/data/apddemographics/[2005-2016].xls


[1/12]: https://raw.githubusercontent.com/utdata/cli-tools/master/data/apddemographics/2005.xls --> 2005.xls
--_curl_--https://raw.githubusercontent.com/utdata/cli-tools/master/data/apddemographics/2005.xls
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  247k  100  247k    0     0  1262k      0 --:--:-- --:--:-- --:--:-- 1482k

[2/12]: https://raw.githubusercontent.com/utdata/cli-tools/master/data/apddemographics/2006.xls --> 2006.xls
--_curl_--https://raw.githubusercontent.com/utdata/cli-tools/master/data/apddemographics/2006.xls
100  249k  100  249k    0     0  2786k      0 --:--:-- --:--:-- --:--:-- 2786k

[3/12]: https://raw.githubusercontent.com/utdata/cli-tools/master/data/apddemographics/2007.xls --> 2007.xls
--_curl_--https://raw.githubusercontent.com/utdata/cli-tools/master/data/apddemographics/2007.xls
100  254k  100  254k    0     0  2020k      0 --:--:-- --:--

In [4]:
# Let's check that we got all the files. There should be 12 of them.
ls

2005.xls	2008.xls	2011.xls	2014.xls
2006.xls	2009.xls	2012.xls	2015.xls
2007.xls	2010.xls	2013.xls	2016.xls


## Inspect one of the files

You should go ahead and open the 2005.xls file in Excel and take a look at it.

- There are multiple worksheets in each file
- Each file is set up exactly the same, with the same fields and the same worksheet names. This would fail if they were not that way. (I actually standardized all these files before this assignment. They were close, but not perfect.)

In [5]:
# let's change our directory to come out of the data directory
# so we can process all these files

# cd up one directory
cd ../

# then check where we are. We should be in the apd directory
pwd

/Users/christian/Documents/rwd/apd


## Convert worksheets into csv files
This command below is a bash loop that uses the csvkit command called [`in2csv`](https://csvkit.readthedocs.io/en/0.9.1/tutorial/1_getting_started.html#in2csv-the-excel-killer) to do a lot of processing of files all at once. There are 12 files, and they each have six worksheets that we want. This is like going into each file, then doing "save as" to make CSVs for each of the six worksheets. So 6 x 12 is 72 ... so 72 "save as" operations in one fell swoop.

- `for` says we are starting loop, meaning we are going to repeat what is in `do` later.
- `i` is a variable that is our marker for the worksheet names that follow.
- `in` is setting up the array of worksheet names. We'll `do` everyhing for "AsstChief", then we'll come back to the top and `do` everything again, but for "Cmdr", etc.

In [7]:
# Convert the worksheets of each file
# It's probably best to copy and paste this into a new cell

# set up loop with vars for each worksheet name in the files
for s in AsstChief Cmdr Lt Sgt Det Corp PO Cadets

# convert into csv for each sheet
do
    in2csv data/2005.xls --sheet $s > data-done/2005-$s.csv
    in2csv data/2006.xls --sheet $s > data-done/2006-$s.csv
    in2csv data/2007.xls --sheet $s > data-done/2007-$s.csv
    in2csv data/2008.xls --sheet $s > data-done/2008-$s.csv
    in2csv data/2009.xls --sheet $s > data-done/2009-$s.csv
    in2csv data/2010.xls --sheet $s > data-done/2010-$s.csv
    in2csv data/2011.xls --sheet $s > data-done/2011-$s.csv
    in2csv data/2012.xls --sheet $s > data-done/2012-$s.csv
    in2csv data/2013.xls --sheet $s > data-done/2013-$s.csv
    in2csv data/2014.xls --sheet $s > data-done/2014-$s.csv
    in2csv data/2015.xls --sheet $s > data-done/2015-$s.csv
    in2csv data/2016.xls --sheet $s > data-done/2016-$s.csv
done

## Breaking down the in2csv command

- `in2csv` is the command we are using. It converts various tabular formats to a csv file.
- data/20XX.xls is the file name we are working on.
- --sheet is a parameter of `in2csv` that says process a specific worksheet in the file.
- `$1` is the worksheet name we are working on in this pass. So the first pass would be "AsstChief", and the next pass would be "Cmdr".
- `>` says take the result of the previous command and write it into a new file.
- `data-done/20XX-$1.csv` is the new file name, but we are using the worksheet name as part of the file. So the first line of the first pass would be `2005-AsstChief.csv`.

## Look at the files you created

If you opened up that folder on your computer while this was running, you could watch the 96 files being created. You could even open them in Atom or Sublime (or even Excel) to see what they look like. There is one file for every worksheet in every file.

We'll just make sure they are there by listing the directory.

In [8]:
# list files in data-done
ls data-done

2005-AsstChief.csv	2009-AsstChief.csv	2013-AsstChief.csv
2005-Cadets.csv		2009-Cadets.csv		2013-Cadets.csv
2005-Cmdr.csv		2009-Cmdr.csv		2013-Cmdr.csv
2005-Corp.csv		2009-Corp.csv		2013-Corp.csv
2005-Det.csv		2009-Det.csv		2013-Det.csv
2005-Lt.csv		2009-Lt.csv		2013-Lt.csv
2005-PO.csv		2009-PO.csv		2013-PO.csv
2005-Sgt.csv		2009-Sgt.csv		2013-Sgt.csv
2006-AsstChief.csv	2010-AsstChief.csv	2014-AsstChief.csv
2006-Cadets.csv		2010-Cadets.csv		2014-Cadets.csv
2006-Cmdr.csv		2010-Cmdr.csv		2014-Cmdr.csv
2006-Corp.csv		2010-Corp.csv		2014-Corp.csv
2006-Det.csv		2010-Det.csv		2014-Det.csv
2006-Lt.csv		2010-Lt.csv		2014-Lt.csv
2006-PO.csv		2010-PO.csv		2014-PO.csv
2006-Sgt.csv		2010-Sgt.csv		2014-Sgt.csv
2007-AsstChief.csv	2011-AsstChief.csv	2015-AsstChief.csv
2007-Cadets.csv		2011-Cadets.csv		2015-Cadets.csv
2007-Cmdr.csv		2011-Cmdr.csv		2015-Cmdr.csv
2007-Corp.csv		2011-Corp.csv		2015-Corp.csv
2007-Det.csv		2011-Det.csv		2015-Det.csv
2007-Lt.csv		2011-Lt.csv		2015-Lt.csv
2007-PO.csv		2011-PO

## Combining the files into one

Our next trick is to use the csvkit command called `csvstack` to combine all the small csv files into a single combined file. This works because they all have the same header row.

Let's break it down before we run it:

- `csvstack` is the command. [Read about it here](https://csvkit.readthedocs.io/en/0.9.1/tutorial/3_power_tools.html#csvstack-combining-subsets).
- `--filenames` is a parameter that adds the filename as a column in each row. This way we know from where each row came. We need this so we know which year the data came from.
- `data-done/20*.csv` is the files we want to combine. We could list them all, but I used a shortcut by using the wildcard * to grab all the files that start with "20" and end with ".csv".
- `> data-done/combined.csv` says to take the output of the csvstack command and put it into a new file by this name.

In [9]:
# Next is to CSVstack them using the filename as a grouping value
# I can use the filename to pull ou the year later in OpenRefine.

csvstack --filenames data-done/20*.csv > apd-demographics-2005-2016.csv

In [10]:
# We can get a line count on the finished file:
wc apd-demographics-2005-2016.csv

   19352   39496 1553486 apd-demographics-2005-2016.csv


In [11]:
# Let's peek at the top of the file:
head apd-demographics-2005-2016.csv

group,Last Name,First Name,Middle Name,Gender,Ethnicity Code,Ethnicity Description,Job Title,Current Hire Date,Commission Date
2005-AsstChief.csv,Ellison,Cathy,Joan,F,2,Black,Assistant Police Chief,1978-08-07,1979-02-23
2005-AsstChief.csv,Coy,Ricky,L,M,1,White,Assistant Police Chief,1975-02-07,1976-11-21
2005-AsstChief.csv,Dahlstrom,Robert,Eric,M,1,White,Assistant Police Chief,1977-07-18,1978-02-24
2005-AsstChief.csv,McDonald,Michael,Charles,M,2,Black,Assistant Police Chief,1983-08-29,1984-08-17
2005-AsstChief.csv,Landeros,Rudy,Garza,M,3,Hispanic,Assistant Police Chief,1981-10-05,1982-04-16
2005-Cadets.csv,Egan,Rae,Ann,F,1,White,Police Cadet,2004-11-29,2004-11-29
2005-Cadets.csv,Swarthout,Tracy,A,F,1,White,Police Cadet,2004-11-29,2004-11-29
2005-Cadets.csv,Castillo,Trissey,Ann,F,3,Hispanic,Police Cadet,2004-11-29,2004-11-29
2005-Cadets.csv,Medrano,Lori,J,F,3,Hispanic,Police Cadet,2004-11-29,2004-11-29


In [12]:
# and the bottom of the file:
tail apd-demographics-2005-2016.csv

2016-Sgt.csv,Torres,Santiago,,M,3,Hispanic,Police Sergeant,2000-10-23,2001-05-25
2016-Sgt.csv,Trejo,Alfred,Louis,M,3,Hispanic,Police Sergeant,1994-07-11,1995-01-06
2016-Sgt.csv,Urias,Steve,,M,3,Hispanic,Police Sergeant,1989-06-05,1989-11-17
2016-Sgt.csv,Vallejo,Carlos,J,M,3,Hispanic,Police Sergeant,2001-03-11,2001-10-19
2016-Sgt.csv,Villanueva,Randy,W.,M,3,Hispanic,Police Sergeant,1998-04-13,1998-10-23
2016-Sgt.csv,Wright,Matthew,,M,3,Hispanic,Police Sergeant,2000-10-23,2001-05-25
2016-Sgt.csv,Yates,Carl,Kevin,M,4.0,American Indian/Aleutian,Police Sergeant,1990-08-06,1991-02-01
2016-Sgt.csv,Bauzon,Jerome,Cortez,M,5,Asian/Pacific Islander,Police Sergeant,1993-07-12,1994-01-07
2016-Sgt.csv,Lee,Shane,Ying,M,5,Asian/Pacific Islander,Police Sergeant,1992-03-23,1992-09-16
2016-Sgt.csv,Rohre,Charles,A,M,5,Asian/Pacific Islander,Police Sergeant,1999-01-04,1999-07-09


## Open the finished file

Now if you go open the `combined.csv` file, you'll see 19,353 rows of data that all came from our 96 files, which were all extracted from our 12 Excel files.

## Feeling lucky?

If everything went right, you can delete your `apd` folder, then go back to the top of the workbook, then go under to the Cell > Run All menu and it will do all your steps again, and you'll have your finished file.

## What's next with this analysis?

When I worked on this file, I next pulled it into Open Refine and I used the filename column to create a "Year" column, along with some other little cleanup here and there.

I then exported that and pulled it into Tableau so I could see how demographics changed in APD over the years.

What questions would you ask this data?

## Turn in your files

- Save your jupyter notebook that you are working on
- close the windows
- Go to your terminal and do Control-c, then say 'yes' to shutting down the server
- upload your .ipynb file and your finished csv "apd-demographics-2005-2016.csv" to the assignment.